In [1]:
import LeakDetector
import numpy as np
import pandas as pd
from log_analysis import (load_dns_data, load_timestamps_from_crawl_data,
                          get_distinct_tcp_conns,
                          get_crawl_status, get_http_df, get_http2_df)

from os.path import isdir, join, basename
from datetime import datetime
from glob import glob

AMAZON_CRAWL = 'amazon-data-20190415-124534'
ROKU_CRAWL = 'roku-data-20190412-122224'

ROOT_CRAWL_DIR = '/mnt/iot-house/crawl-data/'
if not isdir(ROOT_CRAWL_DIR):
    # ROOT_CRAWL_DIR = '/media/gacar/Data/iot-house/crawl-data/'
    ROOT_CRAWL_DIR = '/home/gacar/dev/smart-tv/data'

crawl_data_dir_amazon = join(ROOT_CRAWL_DIR, AMAZON_CRAWL)
crawl_data_dir_roku = join(ROOT_CRAWL_DIR, ROKU_CRAWL)

crawl_data_dirs = {
    "Amazon": crawl_data_dir_amazon,
    "Roku": crawl_data_dir_roku,
}

In [2]:
# TODO: update the IDs
ROKU_IDS = {
    "Serial No": "YG0080901841",
    "AD ID": "ded0f0e3-b3aa-59a2-a143-f6c1157a7ae8",
    "Device ID": "C33858901841",
    "MAC": "d8:31:34:22:e6:ff",
    "City": "Princeton",
    "State": "New Jersey",
    "Zip": "08540",
    "Email": "macyli47@gmail.com",
    "Password": "RoheuskEdfekJa3",
    "Build Number": "519.00E04142A",
    "Device Name": "Office tv",
    "Wifi SSID": "IoT-Pi-3-2",
}

# TODO: ADD AD ID
AMAZON_IDS = {
    "MAC": "68:9a:87:20:3d:1d",
    "AD ID": "05baacb6-acdf-4e18-84cd-ae78eab7e081",
    "Device name": "Macy's 4th Fire TV Stick",
    "Amazon account": "Macy Li",
    "Serial No": "G070L82185152J8P",
    "City": "Princeton",
    "State": "New Jersey",
    "Zip": "08540",
    "Software version": "Fire OS 5.2.6.9 (6325552020)",
    "Email": "macyli47@gmail.com",
    "Password": "M888777",
    "Wifi SSID": "IoT-Pi-1",
}

detector_roku = LeakDetector.LeakDetector(
    ROKU_IDS.values(), encoding_set=LeakDetector.ENCODINGS_NO_ROT,
    encoding_layers=2, hash_layers=2, debugging=False
)


In [3]:
requests_roku, responses_roku, dns_df_roku = get_http_df(crawl_data_dir_roku)
requests_amazon, responses_amazon, dns_df_amazon = get_http_df(crawl_data_dir_amazon)

('Will load HTTP dataframe for', '/home/gacar/dev/smart-tv/data/roku-data-20190412-122224')


/home/gacar/.local/lib/python2.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


('Multiple messages', 402)
('Will load HTTP dataframe for', '/home/gacar/dev/smart-tv/data/amazon-data-20190415-124534')
('Multiple messages', 47)
('Decode errors', 103)
('channels', set(['com.dotstudioz.dotstudioPRO.abstv']))


In [4]:
print(len(requests_roku), len(requests_amazon))

(34345, 14589)


In [5]:
requests_roku.head(3)

,channel_id,time,cookie,post_data,host,referer,url,method,user_agent,ip_dst,tcp_dstport,tcp_stream,http2,http2_type,host_by_dns,channel_name,rank,category,decoded_data
0,73376,1555110679.099763000,,,b.scorecardresearch.com,,http://b.scorecardresearch.com/p2?c1=19&c2=153...,GET,Roku/DVP-9.0 (519.00E04142A),204.153.49.131,80,65,False,,b.scorecardresearch.com,ABC,34,movies-tvs,
1,73376,1555110690.626828000,,,b.scorecardresearch.com,,http://b.scorecardresearch.com/p2?c1=19&c2=153...,GET,Roku/DVP-9.0 (519.00E04142A),204.153.49.131,80,86,False,,b.scorecardresearch.com,ABC,34,movies-tvs,
2,73376,1555110702.181658000,,,b.scorecardresearch.com,,http://b.scorecardresearch.com/p2?c1=19&c2=153...,GET,Roku/DVP-9.0 (519.00E04142A),204.153.49.131,80,110,False,,b.scorecardresearch.com,ABC,34,movies-tvs,


In [6]:
responses_roku.head(3)

,channel_id,data,time,body,host,location,code,set_cookie,ip_src,tcp_srcport,tcp_stream,http2,host_by_dns,channel_name,rank,category
0,73376,01d67b2269223a2266333334393630642d323166662d34...,1555110644.038669000,,,,,,18.207.190.165,2350,17,False,,ABC,34,movies-tvs
1,73376,,1555110679.119502000,GIF89a,,,200,,204.153.49.131,80,65,False,b.scorecardresearch.com,ABC,34,movies-tvs
2,73376,,1555110690.642272000,GIF89a,,,200,,204.153.49.131,80,86,False,b.scorecardresearch.com,ABC,34,movies-tvs


In [7]:
requests_amazon.head(3)

,channel_id,time,cookie,post_data,host,referer,url,method,user_agent,ip_dst,tcp_dstport,tcp_stream,http2,http2_type,host_by_dns,channel_name,rank,category,decoded_data
0,com.amazon.rialto.cordova.webapp.webapp45ae33c...,1555405965.682210000,,,globalstreamingserver.com,,http://globalstreamingserver.com/stv/fire-tv/b...,GET,Mozilla/5.0 (Linux; Android 5.1.1; AFTT Build/...,107.180.50.228,80,3,False,,globalstreamingserver.com,Brick STV Channel,731,,
1,com.amazon.rialto.cordova.webapp.webapp45ae33c...,1555405965.914099000,,,globalstreamingserver.com,http://globalstreamingserver.com/stv/fire-tv/b...,http://globalstreamingserver.com/stv/fire-tv/b...,GET,Mozilla/5.0 (Linux; Android 5.1.1; AFTT Build/...,107.180.50.228,80,4,False,,globalstreamingserver.com,Brick STV Channel,731,,
2,com.amazon.rialto.cordova.webapp.webapp45ae33c...,1555405965.917194000,,,globalstreamingserver.com,http://globalstreamingserver.com/stv/fire-tv/b...,http://globalstreamingserver.com/stv/fire-tv/b...,GET,Mozilla/5.0 (Linux; Android 5.1.1; AFTT Build/...,107.180.50.228,80,5,False,,globalstreamingserver.com,Brick STV Channel,731,,


In [8]:
responses_amazon.head(3)

,channel_id,data,time,body,location,code,set_cookie,ip_src,tcp_srcport,tcp_stream,http2,host_by_dns,channel_name,rank,category,content_type,http2_type,set-cookie
0,com.amazon.rialto.cordova.webapp.webapp45ae33c...,,1555405965.058993000,<html>\n <head>\n <t...,,502,,52.216.176.195,443,1,False,amazon-web-app-whitelist.s3.amazonaws.com,Brick STV Channel,731,,,,
1,com.amazon.rialto.cordova.webapp.webapp45ae33c...,,1555405965.781479000,<!DOCTYPE html>\n\n<!-- \n Web App Starter K...,,200,,107.180.50.228,80,3,False,globalstreamingserver.com,Brick STV Channel,731,,,,
2,com.amazon.rialto.cordova.webapp.webapp45ae33c...,,1555405966.057351000,/* Dialog View\n *\n * Handles modal dialog\n ...,,200,,107.180.50.228,80,8,False,globalstreamingserver.com,Brick STV Channel,731,,,,


In [98]:
CHECK_REFERRER_LEAKS = True

def check_row_for_leaks(detector, req):
    url, cookie_str, post_body, referrer_str = req['url'], req['cookie'], req['post_data'], req['referer']
    url_leaks = detector.check_url(url)
    url_leaks += detector.substring_search(url, max_layers=2)
    #cookie_leaks = detector.check_cookies(headers)
    #cookie_str = detector.get_cookie_str(headers, from_request=True)
    cookie_leaks = detector.substring_search(cookie_str, max_layers=2)
    post_leaks = detector.substring_search(post_body, max_layers=2)
    if CHECK_REFERRER_LEAKS:
        referrer_leaks = detector.substring_search(referrer_str, max_layers=2)
        return url_leaks, cookie_leaks, post_leaks, referrer_leaks
    else:
        return url_leaks, cookie_leaks, post_leaks

In [99]:
def reverse_dict(d):
    new_d = dict()
    for k, v in d.items():
        new_d[v.lower()] = k
    return new_d

In [100]:
def convert_leaks_to_df(device_ids, leaks_dict):
    r_ids = reverse_dict(device_ids)
    leaks_dicts = []
    for leak_type, leaks in leaks_dict.items():
        for leak in leaks:            
            # print(leak, len(leak), leak[0])
            assert len(leak) <= 2
            if len(leak) == 2:
                encoding, search = leak
            elif len(leak) == 1:
                search = leak[0]
                encoding = "unencoded"
            id_type = r_ids[search.lower()]
            leaks_dicts.append({'id_type': id_type, 'search': search,
                                "encoding": encoding, "leak_type": leak_type})
    return pd.DataFrame(leaks_dicts)



In [101]:
def detect_leaks_in_requests(df, device_ids):
    df.sort_values("channel_name", inplace=True)
    last_channel = ""
    leak_df = pd.DataFrame({})
    #for idx, req in df[df.channel_name.str.contains("Z")].iterrows():
    for idx, req in df.iterrows():
        #print(idx, req['request_full_uri'], req['cookie'], req['post_data'], req['referer'], req['data'])
        channel_name = str(req['channel_name'])
        channel_id = str(req['channel_id'])
        if channel_name !=last_channel:
            last_channel = channel_name
            # print("Channel name", channel_name)
            #device_ids = {}
            device_ids["Channel name"] = channel_name
            r_device_ids = reverse_dict(device_ids)
            leak_detector = LeakDetector.LeakDetector(
                device_ids.values(), encoding_set=LeakDetector.ENCODINGS_NO_ROT,
                encoding_layers=2, hash_layers=2, debugging=False
            )
        url_leaks, cookie_leaks, post_leaks, referrer_leaks = check_row_for_leaks(leak_detector, req)
        tmp_df = convert_leaks_to_df(device_ids,
                                     {"url_leaks": url_leaks, "cookie_leaks": cookie_leaks,
                                      "post_leaks": post_leaks, "referrer_leaks": referrer_leaks})
        tmp_df['channel_id'] = channel_id
        tmp_df['channel_name'] = channel_name
        tmp_df['url'] = req['url']
        tmp_df['cookie'] = req['cookie']
        tmp_df['post_data'] = req['post_data']
        tmp_df['referer'] = req['referer']
        leak_df = leak_df.append(tmp_df)
        #if len(url_leaks) or len(cookie_leaks) or len(post_leaks) or len(referrer_leaks):
        #    if "scorecard" not in req['url']:
        #        print (req['url'], req['cookie'], req['post_data'], req['referer'], url_leaks, cookie_leaks, post_leaks, referrer_leaks)
    device_ids["Channel name"] = ""
    return leak_df


## Roku leaks

In [ ]:
leak_df_roku = detect_leaks_in_requests(requests_roku, ROKU_IDS)

In [ ]:
leak_df_roku.head()

In [ ]:
leak_df_roku.id_type.value_counts()

In [ ]:
leak_df_roku.leak_type.value_counts()

In [ ]:
leak_df_roku.encoding.value_counts()

In [ ]:
leak_df_roku[leak_df_roku.id_type=="Channel name"].channel_id.nunique()

In [ ]:
leak_df_roku[leak_df_roku.id_type=="AD ID"].channel_id.nunique()

In [ ]:
leak_df_roku[leak_df_roku.id_type=="Build Number"].channel_id.nunique()

In [ ]:
leak_df_roku[leak_df_roku.id_type=="Serial No"].channel_id.nunique()

In [ ]:
leak_df_roku[leak_df_roku.id_type=="City"].channel_id.nunique()

In [ ]:
leak_df_roku[leak_df_roku.id_type=="Zip"].channel_id.nunique()

In [ ]:
leak_df_roku[leak_df_roku.id_type=="Device Name"].channel_id.nunique()

## Amazon leaks
- TODO add AD ID

In [90]:
leak_df_amazon = detect_leaks_in_requests(requests_amazon, AMAZON_IDS)

In [91]:
leak_df_amazon.id_type.value_counts()

Channel name    2834
City              30
Zip               26
State             13
Name: id_type, dtype: int64

In [92]:
leak_df_amazon.leak_type.value_counts()

url_leaks         2473
referrer_leaks     318
cookie_leaks        80
post_leaks          32
Name: leak_type, dtype: int64

In [93]:
leak_df_amazon.encoding.value_counts()

unencoded    2890
urlencode      13
Name: encoding, dtype: int64